## Установка pandas

In [2]:
# !pip3 install pandas

In [3]:
import pandas as pd

## Чтение данных и фильтрация таблиц

In [35]:
user = pd.read_csv('datasets/sirius/user.csv')
user.head()

,id,gender,birthday,grade,region,last_visited
0,1,М,02.11.2007,9.0,Город Санкт-Петербург,23.08.2023 03:04:14
1,2,Ж,29.01.2009,8.0,Город Москва,07.09.2023 09:21:35
2,3,М,09.05.2007,NaN,Город Санкт-Петербург,23.08.2023 18:32:18
3,4,Ж,27.09.2007,9.0,Город Москва,18.08.2023 07:34:11
4,5,Ж,04.08.2007,NaN,Город Москва,02.09.2023 18:31:30


### Задание 1

Сколько в таблице `user` представлено учеников, родившихся в сентябре?

In [27]:
bdates = pd.to_datetime(user.birthday, format='%d.%m.%Y')
(user.birthday[bdates.dt.month == 9]).count()

73

### Задание 2

Какое количество учеников в последний раз посещали платформу в первые $14$ дней сентября?

In [30]:
ldates = pd.to_datetime(user.last_visited, format='%d.%m.%Y %H:%M:%S')
(user.last_visited[(ldates.dt.month == 9) & (ldates.dt.day <= 14)]).count()

377

### Задание 3

На сколько в данных больше девятиклассников, чем девятиклассниц?

In [46]:
# user.grade[user.grade.isna()] = 0
# user.grade = user.grade.astype('int')
# user.grade

In [48]:
# user.head()
(user.gender[(user.gender == 'М') & (user.grade == 9)]).count() - (user.gender[(user.gender == 'Ж') & (user.grade == 9)]).count()

5

### Задание 4

Какой процент учеников, у которых указан класс, обучается в $9$-м классе?

Ответ округлите до целых.

In [67]:
(100 * user.grade[user.grade == 9].count() / user.grade[user.grade != 0].count()).round()

24.0

### Задание 5

Сколько в данных представлено учеников из регионов, в названиях которых в качестве подстроки содержится слово «город»?

При расчёте статистики приведите названия регионов к строчному написанию.

In [68]:
user.head()

,id,gender,birthday,grade,region,last_visited
0,1,М,02.11.2007,9,Город Санкт-Петербург,23.08.2023 03:04:14
1,2,Ж,29.01.2009,8,Город Москва,07.09.2023 09:21:35
2,3,М,09.05.2007,0,Город Санкт-Петербург,23.08.2023 18:32:18
3,4,Ж,27.09.2007,9,Город Москва,18.08.2023 07:34:11
4,5,Ж,04.08.2007,0,Город Москва,02.09.2023 18:31:30


In [74]:
user.region[user.region.str.lower().str.contains('город')].count()

765

## Группировка, агрегация и соединение таблиц

In [123]:
course = pd.read_csv('datasets/sirius/course.csv')

In [124]:
user_course_progress = pd.read_csv('datasets/sirius/user_course_progress.csv')

**Относительным прогрессом** будем называть отношение суммарного числа баллов, которые ученик набрал за курс (колонка `progress` в таблице `user_course_progress`), к максимальному количеству баллов, которые ученик в принципе мог набрать за данный курс (колонка `max_progress` в таблице `course`).

С помощью представленных данных найдите ответ на следующий вопрос: в курсах какого предмета больше всего учеников, чей относительный прогресс не меньше $0.8$?

В рамках следующих заданий вам будет предложено, действуя поэтапно, найти ответ на этот вопрос.

### Задание 1

Произведите соединение таблиц `course` и `user_course_progress`.

В качестве ответа укажите количество строчек итоговой таблицы, в которых представлены ученики курсов по химии.

In [126]:
user_courses = user_course_progress \
    .set_index('user_id').join(user.set_index('id')) \
    .set_index('course_id').join(course.set_index('id')) \
    .drop(['gender', 'birthday', 'region', 'last_visited', 'date_started'], axis=1)
user_courses.head()

,progress,grade,subject,max_score
course_id,,,,
1,22.64,9,Химия,108.0
1,95.62,10,Химия,108.0
1,14.62,9,Химия,108.0
1,26.46,10,Химия,108.0
1,32.32,0,Химия,108.0


In [127]:
user_courses.subject[user_courses.subject == 'Химия'].count()

678

### Задание 2

Обратите внимание, что у части курсов нет значения в колонке `max_score`. Уберите из полученной в предыдщуем задании таблицы все строчки, в которых пропущено значение колонки `max_score`.

В качестве ответа укажите количество строчек в итоговой таблице.

In [128]:
user_courses = user_courses.dropna()
len(user_courses)

1031

### Задание 3

Добавьте в полученную в предыдущем задании таблицу колонку `relative_progress`, в которой для каждого ученика будет храниться значение его относительного прогресса в конкретном курсе.

В качестве ответа укажите среднее значение относительного прогресса в таблице. Ответ округлите до двух знаков после запятой с помощью функции `round`, которая встроена в Series.

In [129]:
user_courses['relative_progress'] = user_courses.progress / user_courses.max_score
user_courses.head()

,progress,grade,subject,max_score,relative_progress
course_id,,,,,
1,22.64,9,Химия,108.0,0.209630
1,95.62,10,Химия,108.0,0.885370
1,14.62,9,Химия,108.0,0.135370
1,26.46,10,Химия,108.0,0.245000
1,32.32,0,Химия,108.0,0.299259


In [131]:
user_courses.relative_progress.mean().round(2)

0.35

### Задание 4

Оставьте в полученной в предыдущем задании таблице только те строчки, в которых значение относительного прогресса не меньше $0.8$.

В качестве ответа укажите число строчек в получившейся таблице.

In [143]:
good_user_courses = user_courses[user_courses.relative_progress >= 0.8]
len(good_user_courses)

159

### Задание 5

Произведите группировку строчек таблицы по предметам и посчитайте, сколько учеников с относительным прогрессом не меньше $0.8$ оказалось в каждом из предметов.

В качестве ответа заполните таблицу, в которой каждому предмету сопоставлено число таких учеников.

Если один и тот же ученик учился в разных курсах одного и того же предмета и в каждом из этих курсов достиг относительного прогресса не менее $0.8$, при подсчёте учтите его несколько раз.

In [144]:
good_user_courses = good_user_courses.drop(['progress', 'grade', 'max_score'], axis=1)
good_user_courses.groupby('subject').count()

,relative_progress
subject,
Математика,30
Физика,24
Химия,105


### Задание 6

В курсах какого предмета больше всего учеников, достигших относительного прогресса не менее $0.8$?

In [172]:
winner = good_user_courses.groupby('subject').count()
print(f"{winner['relative_progress'].idxmax()}: {winner['relative_progress'].max()} человек")

Химия: 105 человек


In [9]:
import pandas as pd
import numpy as np

In [33]:
di = {'id': [0 , 1], 'gender': ['М', 'Ж'], 'age': [12, 31], 'grade': [1, 2], 'region': ['Пр', 'Нр']}
tab = pd.DataFrame(di)

In [35]:
tab

,id,gender,age,grade,region
0,0,М,12,1,Пр
1,1,Ж,31,2,Нр


In [48]:
tab \
    .assign(gender=lambda df: df['gender'].str.replace('М', 'Мужсĸой').str.replace('Ж', 'Женсĸий')) \
    .query('gender == "Мужсĸой"') \
    .drop(columns='gender') \
    .groupby(['region', 'age'], as_index=False) \
    .agg({'id': 'count', 'grade': 'mean'}) \
    .rename(columns={'id': 'count', 'grade': 'avg_grade'}) \
    .assign(count_share=lambda df: df['count'] / df['count'].sum()) \
    

,region,age,count,avg_grade,count_share
0,Пр,12,1,1.0,1.0
